# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("./cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,clyde river,70.4692,-68.5914,-15.87,72,75,12.35,CA,1681302694
1,1,kloulklubed,7.0419,134.2556,26.28,74,100,4.13,PW,1681302697
2,2,dikson,73.5069,80.5464,-19.33,92,96,8.95,RU,1681302698
3,3,busselton,-33.6500,115.3333,18.94,76,100,5.90,AU,1681302699
4,4,yumen,40.2833,97.2000,12.99,30,100,7.88,CN,1681302701


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather = city_data_df.loc[(city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 34) & 
                                  (city_data_df['Wind Speed'] <= 9) & (city_data_df['Cloudiness'] <= 50) & 
                                (city_data_df['Humidity'] <= 70)]

# Display sample data
best_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ponta do sol,32.6667,-17.1000,22.62,69,0,1.84,PT,1681302703
8,8,port elizabeth,-33.9180,25.5701,22.83,69,0,4.63,ZA,1681302704
34,34,los llanos de aridane,28.6585,-17.9182,21.86,60,20,7.72,ES,1681302730
35,35,castro,-24.7911,-50.0119,21.75,69,0,1.94,BR,1681302731
37,37,santa cruz de la palma,28.6835,-17.7642,24.08,60,20,7.72,ES,1681302732
42,42,awbari,26.5921,12.7805,24.59,12,0,1.60,LY,1681302736
59,59,east london,-33.0153,27.9116,21.52,64,0,4.63,ZA,1681302759
62,62,madras,13.0878,80.2785,33.99,68,20,4.12,IN,1681302637
75,75,palasa,18.7667,84.4167,29.39,64,0,2.51,IN,1681302774
90,90,bandarbeyla,9.4942,50.8122,29.22,67,24,5.16,SO,1681302788


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather.copy()
hotel_df = hotel_df.drop(columns=["City_ID","Cloudiness", "Wind Speed", "Max Temp", "Date"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
7,ponta do sol,32.6667,-17.1000,69,PT,
8,port elizabeth,-33.9180,25.5701,69,ZA,
34,los llanos de aridane,28.6585,-17.9182,60,ES,
35,castro,-24.7911,-50.0119,69,BR,
37,santa cruz de la palma,28.6835,-17.7642,60,ES,
42,awbari,26.5921,12.7805,12,LY,
59,east london,-33.0153,27.9116,64,ZA,
62,madras,13.0878,80.2785,68,IN,
75,palasa,18.7667,84.4167,64,IN,
90,bandarbeyla,9.4942,50.8122,67,SO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ponta do sol - nearest hotel: Estalagem Ponta do Sol
port elizabeth - nearest hotel: Waterford Hotel
los llanos de aridane - nearest hotel: Valle Aridane
castro - nearest hotel: Hotel Central Palace
santa cruz de la palma - nearest hotel: El Hotelito 27
awbari - nearest hotel: فندق اوباري
east london - nearest hotel: No hotel found
madras - nearest hotel: Blue Star International
palasa - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
cam ranh - nearest hotel: Khách sạn Thái Bình


,City,Lat,Lng,Humidity,Country,Hotel Name
7,ponta do sol,32.6667,-17.1000,69,PT,Estalagem Ponta do Sol
8,port elizabeth,-33.9180,25.5701,69,ZA,Waterford Hotel
34,los llanos de aridane,28.6585,-17.9182,60,ES,Valle Aridane
35,castro,-24.7911,-50.0119,69,BR,Hotel Central Palace
37,santa cruz de la palma,28.6835,-17.7642,60,ES,El Hotelito 27
42,awbari,26.5921,12.7805,12,LY,فندق اوباري
59,east london,-33.0153,27.9116,64,ZA,No hotel found
62,madras,13.0878,80.2785,68,IN,Blue Star International
75,palasa,18.7667,84.4167,64,IN,No hotel found
90,bandarbeyla,9.4942,50.8122,67,SO,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)